# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [5]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a investigative jouenalist who is an expert at getting insights for a story. you write what is currently interesting about a given topic and find interesting information: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to get interesting information",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for planning the investigation",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_50920/1675081462.py:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)


In [6]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
print(tool_names)
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

['Search', 'TODO']


/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_50920/94880364.py:5: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~create_react_agent` instead.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [7]:
OBJECTIVE = "Write a 300 word investigative teaser of North korea's invovlement in the Ukrain war"

In [8]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [9]:
baby_agi({"objective": OBJECTIVE})

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_50920/3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: To investigate North Korea's involvement in the Ukraine war, I should first make a list of tasks that need to be completed.
Action: TODO
Action Input: callbacks=[Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, Search, S

{'objective': "Write a 300 word investigative teaser of North korea's invovlement in the Ukrain war"}